##Objective - To perform an analysis on some categories of the News to find out how there is a shift in the sentiment over 10 years

In [2]:
import pandas as pd
import numpy as np
import pickle
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from torch.cuda.amp import autocast

In [ ]:
df = pd.read_json('News_data.json',lines=True)

#print(df)
df['Year'] = pd.DatetimeIndex(df['date']).year
df['Month'] = pd.DatetimeIndex(df['date']).month

In [ ]:
df_2012 = df[df['Year']==2012]
df_2013 = df[df['Year']==2013]
df_2014 = df[df['Year']==2014]
df_2015 = df[df['Year']==2015]
df_2016 = df[df['Year']==2016]
df_2017 = df[df['Year']==2017]
df_2018 = df[df['Year']==2018]
df_2019 = df[df['Year']==2019]
df_2020 = df[df['Year']==2020]
df_2021 = df[df['Year']==2021]
df_2022 = df[df['Year']==2022]

df_2012.to_csv('/content/News_2012.csv',index=False)
df_2013.to_csv('/content/News_2013.csv',index=False)
df_2014.to_csv('/content/News_2014.csv',index=False)
df_2015.to_csv('/content/News_2015.csv',index=False)
df_2016.to_csv('/content/News_2016.csv',index=False)
df_2017.to_csv('/content/News_2017.csv',index=False)
df_2018.to_csv('/content/News_2018.csv',index=False)
df_2019.to_csv('/content/News_2019.csv',index=False)
df_2020.to_csv('/content/News_2020.csv',index=False)
df_2021.to_csv('/content/News_2021.csv',index=False)
df_2022.to_csv('/content/News_2022.csv',index=False)

In [4]:
df_2012 = pd.read_csv('/content/News_2012.csv')
df_2013 = pd.read_csv('/content/News_2013.csv')
df_2014 = pd.read_csv('/content/News_2014.csv')
df_2015 = pd.read_csv('/content/News_2015.csv')
df_2016 = pd.read_csv('/content/News_2016.csv')
df_2017 = pd.read_csv('/content/News_2017.csv')
df_2018 = pd.read_csv('/content/News_2018.csv')
df_2019 = pd.read_csv('/content/News_2019.csv')
df_2020 = pd.read_csv('/content/News_2020.csv')
df_2021 = pd.read_csv('/content/News_2021.csv')
df_2022 = pd.read_csv('/content/News_2022.csv')

In [5]:
df_2012.dropna(subset=['headline'],inplace=True)
df_2013.dropna(subset=['headline'],inplace=True)
df_2014.dropna(subset=['headline'],inplace=True)
df_2015.dropna(subset=['headline'],inplace=True)
df_2016.dropna(subset=['headline'],inplace=True)
df_2017.dropna(subset=['headline'],inplace=True)
df_2018.dropna(subset=['headline'],inplace=True)
df_2019.dropna(subset=['headline'],inplace=True)
df_2020.dropna(subset=['headline'],inplace=True)
df_2021.dropna(subset=['headline'],inplace=True)
df_2022.dropna(subset=['headline'],inplace=True)

In [ ]:
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Move the model to the specified device
model = model.to(device)


#Function for finding regression scores of sentiment

def _extract_sentiment_score(text_list):
    # Assuming that the max length of the headlines will not cross 40 tokens
    sentiment_scores = []

    with torch.no_grad(), autocast():
        for text_batch in text_list:
            inputs = tokenizer(text_batch, return_tensors="pt",
                               padding='max_length',
                               max_length=50).to(torch.device('cuda'))
            logits = model(**inputs).logits

            # Capture the class and the regression score from the logits
            # Class 0 means NEGATIVE and 1 means POSITIVE
            predicted_class_reg_score = logits.max(dim=1)

            # Append the class and the scores as a list of tuples
            sentiment_scores.extend(list(zip(predicted_class_reg_score.indices.detach().cpu().numpy(),
                                              predicted_class_reg_score.values.detach().cpu().numpy())))

    return sentiment_scores


#Convert them to positive or negative
def adjust_regression_scores(sentiment_scores):
    adjusted_scores = []
    for class_label, reg_score in sentiment_scores:
        if class_label == 0:  # Negative class
            reg_score *= -1
        adjusted_scores.append(reg_score)
    return adjusted_scores

#adjusted_scores = adjust_regression_scores(_extract_sentiment_score(x))
#print(adjusted_scores)


#test list is the text input to be provided to the function when calling it. Here x is the input text list

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Using device: cuda


In [ ]:
categories = df_2012['category'].unique()
cat_senti_2012 = []
for category in categories:
  headlines = df_2012['headline'][df_2012['category']== category]
  adjusted_scores = adjust_regression_scores(_extract_sentiment_score(headlines))
  #print(round(np.average(adjusted_scores),2))
  cat_senti_2012.append(round(np.average(adjusted_scores),2))

print(cat_senti_2012)
print("Length of Category list",len(categories))
print("Length of senti list",len(cat_senti_2012))

[0.19, 0.1, 0.99, 1.08, 0.75, 1.54, 0.85, -0.8, 0.23, 1.36, -0.22, 0.29, 1.44, -1.12, -0.78, 0.17, 1.69, 0.45, -1.73, 0.56, -1.22]
Length of Category list 21
Length of senti list 21


In [ ]:
categories = df_2013['category'].unique()
cat_senti_2013 = []
for category in categories:
  headlines = df_2013['headline'][df_2013['category']== category]
  adjusted_scores = adjust_regression_scores(_extract_sentiment_score(headlines))
  cat_senti_2013.append(round(np.average(adjusted_scores),2))

print(cat_senti_2013)
print("Length of Category list",len(categories))
print("Length of senti list",len(cat_senti_2013))

[0.16, 0.88, 0.22, -0.36, 1.22, -0.44, 1.71, 0.64, 0.99, -1.56, 0.08, -0.38, 0.41, -1.21, -0.51, 1.13, 0.15, 1.27, 1.79, 0.27, 0.99]
Length of Category list 21
Length of senti list 21


In [ ]:
categories = df_2014['category'].unique()
cat_senti_2014 = []
for category in categories:
  headlines = df_2014['headline'][df_2014['category']== category]
  print(category)
  adjusted_scores = adjust_regression_scores(_extract_sentiment_score(headlines))
  cat_senti_2014.append(round(np.average(adjusted_scores),2))

print(cat_senti_2014)
print("Length of Category list",len(categories))
print("Length of senti list",len(cat_senti_2014))

WOMEN
BUSINESS
STYLE
TASTE
PARENTS
SPORTS
HEALTHY LIVING
COLLEGE
SCIENCE
LATINO VOICES
COMEDY
ARTS
WORLDPOST
POLITICS
ENTERTAINMENT
FIFTY
MEDIA
QUEER VOICES
GREEN
IMPACT
CRIME
BLACK VOICES
RELIGION
GOOD NEWS
TRAVEL
WEIRD NEWS
EDUCATION
WELLNESS
PARENTING
HOME & LIVING
STYLE & BEAUTY
DIVORCE
WEDDINGS
FOOD & DRINK
MONEY
ENVIRONMENT
TECH
CULTURE & ARTS
[0.28, -0.08, 1.24, 1.18, 0.36, 0.37, 0.36, 0.03, 0.25, -0.0, -0.13, 1.76, -0.52, -0.68, 0.85, 0.72, -0.86, 0.36, 0.16, 1.07, -1.92, 0.19, 0.64, 1.62, 1.58, -0.97, 0.63, 0.18, 0.17, 0.84, 1.09, -0.26, 1.33, 0.73, -1.14, -0.05, -0.42, 1.78]
Length of Category list 38
Length of senti list 38


In [ ]:
categories = df_2015['category'].unique()
cat_senti_2015 = []
for category in categories:
  headlines = df_2015['headline'][df_2015['category']== category]
  adjusted_scores = adjust_regression_scores(_extract_sentiment_score(headlines))
  cat_senti_2015.append(round(np.average(adjusted_scores),2))

print(cat_senti_2015)
print("Length of Category list",len(categories))
print("Length of senti list",len(cat_senti_2015))

[-1.93, -0.7, -0.64, -0.31, 0.68, -0.01, -0.94, 0.15, 0.29, 0.11, 0.6, -1.08, 0.4, 1.02, 0.98, 0.95, 0.64, 0.12, -0.03, -0.17, -0.96, 1.27, 0.02, 1.38, 0.96, -0.81, 1.85, 0.85, -0.35, 0.51]
Length of Category list 30
Length of senti list 30


In [ ]:
categories = df_2016['category'].unique()
cat_senti_2016 = []
for category in categories:
  headlines = df_2016['headline'][df_2016['category']== category]
  adjusted_scores = adjust_regression_scores(_extract_sentiment_score(headlines))
  cat_senti_2016.append(round(np.average(adjusted_scores),2))

print(cat_senti_2015)
print("Length of Category list",len(categories))
print("Length of senti list",len(cat_senti_2015))

[-1.93, -0.7, -0.64, -0.31, 0.68, -0.01, -0.94, 0.15, 0.29, 0.11, 0.6, -1.08, 0.4, 1.02, 0.98, 0.95, 0.64, 0.12, -0.03, -0.17, -0.96, 1.27, 0.02, 1.38, 0.96, -0.81, 1.85, 0.85, -0.35, 0.51]
Length of Category list 30
Length of senti list 30


In [ ]:
categories = df_2017['category'].unique()
cat_senti_2017 = []
for category in categories:
  headlines = df_2017['headline'][df_2017['category']== category]
  adjusted_scores = adjust_regression_scores(_extract_sentiment_score(headlines))
  cat_senti_2017.append(round(np.average(adjusted_scores),2))

print(cat_senti_2017)
print("Length of Category list",len(categories))
print("Length of senti list",len(cat_senti_2017))

[-0.97, -1.13, -0.07, -0.39, 0.16, 0.34, 0.36, 0.64, -0.57, -0.83, -1.02, -2.13, -0.0, -0.22, -1.53, -0.52, -1.26, 0.47, -0.01, -0.63, 0.66, 0.78, 0.62, 0.15, -0.91, 0.78, -0.85, 1.71, 0.6, 3.26, 2.67]
Length of Category list 31
Length of senti list 31


In [ ]:
df_2018 = pd.read_csv('/content/News_2018.csv')
categories = df_2018['category'].unique()
cat_senti_2018 = []
for category in categories:
  headlines = df_2018['headline'][df_2018['category']== category]
  adjusted_scores = adjust_regression_scores(_extract_sentiment_score(headlines))
  cat_senti_2018.append(round(np.average(adjusted_scores),2))

print(cat_senti_2018)
print("Length of Category list",len(categories))
print("Length of senti list",len(cat_senti_2018))

[-1.59, -2.6, -0.97, -0.25, -1.14, -1.31, 0.21, -1.07, -2.18, 2.15, -0.8, 0.22, -0.78, -0.52, -1.63, 1.28, -0.16, -2.88, 1.72, -0.58, -0.33, 1.97, 0.19, -2.16, -2.07, 2.21, 1.37, -0.02, -0.66, 1.25, 1.43, 1.71, 0.72, -0.91, -0.38, -0.63]
Length of Category list 36
Length of senti list 36


In [ ]:
categories = df_2019['category'].unique()
cat_senti_2019 = []
for category in categories:
  headlines = df_2019['headline'][df_2019['category']== category]
  adjusted_scores = adjust_regression_scores(_extract_sentiment_score(headlines))
  cat_senti_2019.append(round(np.average(adjusted_scores),2))

print(cat_senti_2019)
print("Length of Category list",len(categories))
print("Length of senti list",len(cat_senti_2019))

[-1.52, -1.36, -1.12, -0.51, 0.61, -0.29, -0.06, -0.32, -2.39, -0.58, 0.93, -0.78, -0.68, -0.1, -0.99, -1.95, -0.85, 0.62, -0.29, -2.65, -0.91, -0.55, -3.49, 3.17, 0.48, 1.57, -1.1, 4.3]
Length of Category list 28
Length of senti list 28


In [ ]:
categories = df_2020['category'].unique()
cat_senti_2020 = []
for category in categories:
  headlines = df_2020['headline'][df_2020['category']== category]
  adjusted_scores = adjust_regression_scores(_extract_sentiment_score(headlines))
  cat_senti_2020.append(round(np.average(adjusted_scores),2))

print(cat_senti_2020)
print("Length of Category list",len(categories))
print("Length of senti list",len(cat_senti_2020))

[-1.54, 0.18, 0.02, -1.46, 0.43, 1.79, -0.02, -2.55, -2.29, -1.1, 0.61, -1.67, -0.44, 0.1, 0.66, -1.59, 1.47, 1.74, -2.03, 0.98, 0.25, -1.77, -1.8, 0.75, -2.37]
Length of Category list 25
Length of senti list 25


In [ ]:
categories = df_2021['category'].unique()
cat_senti_2021 = []
for category in categories:
  headlines = df_2021['headline'][df_2021['category']== category]
  adjusted_scores = adjust_regression_scores(_extract_sentiment_score(headlines))
  cat_senti_2021.append(round(np.average(adjusted_scores),2))

print(cat_senti_2021)
print("Length of Category list",len(categories))
print("Length of senti list",len(cat_senti_2021))

[-0.08, -1.21, 1.11, 0.07, -1.23, -0.69, 0.72, -1.48, 1.2, -1.87, -1.44, -1.24, -0.24, 0.05, -0.57, 1.18, 2.13, -0.84, -0.57, -2.24, -3.57, -3.25, -0.49, -1.95, -2.64, 0.57, -2.19]
Length of Category list 27
Length of senti list 27


In [ ]:
categories = df_2022['category'].unique()
cat_senti_2022 = []
for category in categories:
  headlines = df_2022['headline'][df_2022['category']== category]
  adjusted_scores = adjust_regression_scores(_extract_sentiment_score(headlines))
  cat_senti_2022.append(round(np.average(adjusted_scores),2))

print(cat_senti_2022)
print("Length of Category list",len(categories))
print("Length of senti list",len(cat_senti_2022))

[-1.27, 0.58, 0.77, -1.04, 0.9, -3.18, -0.16, -0.1, -1.25, 0.2, -1.37, 0.55, -2.1, -0.36, -1.5, -2.18, -0.56, 0.3, 1.02, 0.99, 2.72, 3.54, 2.46, 0.92]
Length of Category list 24
Length of senti list 24


In [ ]:
complete_senti = [cat_senti_2012,cat_senti_2013,cat_senti_2014,cat_senti_2015,cat_senti_2016,cat_senti_2017,cat_senti_2018,cat_senti_2019,cat_senti_2020,cat_senti_2021,cat_senti_2022]

In [ ]:
file_path = 'senti.pickle'
with open(file_path,'wb') as f:
  pickle.dump(complete_senti,f)

In [3]:
with open('/content/senti.pickle','rb') as f:
  complete_senti = pickle.load(f)

print(complete_senti)

[[0.19, 0.1, 0.99, 1.08, 0.75, 1.54, 0.85, -0.8, 0.23, 1.36, -0.22, 0.29, 1.44, -1.12, -0.78, 0.17, 1.69, 0.45, -1.73, 0.56, -1.22], [0.16, 0.88, 0.22, -0.36, 1.22, -0.44, 1.71, 0.64, 0.99, -1.56, 0.08, -0.38, 0.41, -1.21, -0.51, 1.13, 0.15, 1.27, 1.79, 0.27, 0.99], [0.28, -0.08, 1.24, 1.18, 0.36, 0.37, 0.36, 0.03, 0.25, -0.0, -0.13, 1.76, -0.52, -0.68, 0.85, 0.72, -0.86, 0.36, 0.16, 1.07, -1.92, 0.19, 0.64, 1.62, 1.58, -0.97, 0.63, 0.18, 0.17, 0.84, 1.09, -0.26, 1.33, 0.73, -1.14, -0.05, -0.42, 1.78], [-1.93, -0.7, -0.64, -0.31, 0.68, -0.01, -0.94, 0.15, 0.29, 0.11, 0.6, -1.08, 0.4, 1.02, 0.98, 0.95, 0.64, 0.12, -0.03, -0.17, -0.96, 1.27, 0.02, 1.38, 0.96, -0.81, 1.85, 0.85, -0.35, 0.51], [-0.86, 0.36, -1.29, -1.07, -0.65, 0.25, -1.21, 0.83, 0.59, 0.23, -0.36, 1.2, 1.01, -1.38, 0.58, -0.87, 0.92, 0.18, -0.25, -0.33, 0.62, -0.8, 1.03, -0.09, -2.15, 0.3, 1.07, 1.73, 0.09, 0.38], [-0.97, -1.13, -0.07, -0.39, 0.16, 0.34, 0.36, 0.64, -0.57, -0.83, -1.02, -2.13, -0.0, -0.22, -1.53, -0.52, -

In [6]:
cat_2012 = df_2012['category'].unique().tolist()
cat_2013 = df_2013['category'].unique().tolist()
cat_2014 = df_2014['category'].unique().tolist()
cat_2015 = df_2015['category'].unique().tolist()
cat_2016 = df_2016['category'].unique().tolist()
cat_2017 = df_2017['category'].unique().tolist()
cat_2018 = df_2018['category'].unique().tolist()
cat_2019 = df_2019['category'].unique().tolist()
cat_2020 = df_2020['category'].unique().tolist()
cat_2021 = df_2021['category'].unique().tolist()
cat_2022 = df_2022['category'].unique().tolist()

In [8]:
common_elements = set(cat_2012).intersection(cat_2013,cat_2014,cat_2015,cat_2016,cat_2017,cat_2018,cat_2019,cat_2020,cat_2021,cat_2022)
print(common_elements)

{'QUEER VOICES', 'CRIME', 'BUSINESS', 'TRAVEL', 'SPORTS', 'COMEDY', 'BLACK VOICES', 'ENTERTAINMENT', 'SCIENCE'}


In [9]:
for i in common_elements:
    locals()[f"{i}"] = [l.index(i) if i in l else None for l in [cat_2013,cat_2013,cat_2014,cat_2015,cat_2016,cat_2017,cat_2018,cat_2019,cat_2020,cat_2021,cat_2022]]

In [10]:
Entertainment = []
for i,j in zip(complete_senti,ENTERTAINMENT):
    Entertainment.append(i[j])

In [11]:
Science = []
for i,j in zip(complete_senti,SCIENCE):
    Science.append(i[j])

In [12]:
Crime = []
for i,j in zip(complete_senti,CRIME):
    Crime.append(i[j])

In [13]:
Business = []
for i,j in zip(complete_senti,BUSINESS):
    Business.append(i[j])

In [14]:
Sports = []
for i,j in zip(complete_senti,SPORTS):
    Sports.append(i[j])

In [15]:
Travel = []
for i,j in zip(complete_senti,TRAVEL):
    Travel.append(i[j])

In [16]:
Comedy = []
for i,j in zip(complete_senti,COMEDY):
    Comedy.append(i[j])

In [ ]:
df = pd.read_json('/content/News_data.json',lines=True)
df['Year'] = pd.DatetimeIndex(df['date']).year
df['Month'] = pd.DatetimeIndex(df['date']).month
df.to_csv('/content/News_data.csv',index=False)

In [ ]:
Years = df['Year'].unique().tolist()
Years

In [ ]:
Years = Years[::-1]
Years

In [ ]:
import matplotlib.pyplot as plt

plt.subplot(2,3,1)
plt.plot(Years,Entertainment)
plt.xlabel('Years')
plt.ylabel('Sentiment')
plt.title('Entertainment')

plt.subplot(2,3,2)
plt.plot(Years,Comedy)
plt.xlabel('Years')
plt.ylabel('Sentiment')
plt.title('Comedy')

plt.subplot(2,3,3)
plt.plot(Years,Travel)
plt.xlabel('Years')
plt.ylabel('Sentiment')
plt.title('Travel')

plt.subplot(2,3,4)
plt.plot(Years,Sports)
plt.xlabel('Years')
plt.ylabel('Sentiment')
plt.title('Sports')

plt.subplot(2,3,5)
plt.plot(Years,Business)
plt.xlabel('Years')
plt.ylabel('Sentiment')
plt.title('Business')

plt.subplot(2,3,6)
plt.plot(Years,Crime)
plt.xlabel('Years')
plt.ylabel('Sentiment')
plt.title('Crime')